# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json
from sql_queries import *
import datetime

In [2]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    print("connection to sparkifydb Database was successfull")
except psycopg2.Error as e:
    print("Connection to the sparkifydb database was not successful")
    print(e)

connection to sparkifydb Database was successfull


In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = str (song_files[0])
filepath

'c:\\Users\\Motunrayo Ibiyo\\Documents\\DE\\Udacity_project1\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json'

In [6]:
"""load the Json file into a list of dictionaries so that it can be converted into a dataframe"""
cumm_song_list = []
for song_datum in song_files:
    with open(song_datum) as song_file:
        content = json.load(song_file)
        cumm_song_list.append(content)

In [7]:
df = pd.DataFrame(cumm_song_list)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id','title','artist_id','year','duration']]
song_data.head()

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
1,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
2,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.47546
3,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
4,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
def generate_filename(data):
    upload_filename =datetime.datetime.now().strftime("%m%d%Y%H%M%S%f")+data
    upload_filename = upload_filename + ".csv"
    uploaded_filepath = "uploaded_data\\" + upload_filename
    return uploaded_filepath

In [10]:
uploaded_filepath = generate_filename("song_table") 
song_data.to_csv(uploaded_filepath, index=False, header=True,sep=',', encoding='utf-8')
new_songs = open(uploaded_filepath)
print('file opened in memory')

file opened in memory


In [11]:
copy_sql = '''COPY public.songs FROM STDIN WITH 
CSV
HEADER
DELIMITER AS ','
'''
try:
    cur.copy_expert(sql=copy_sql,file=new_songs)
    print("file copied to table")
except psycopg2.Error as e:
    print("could not copy file "+ uploaded_filepath+ " to the table")
    print(e)

file copied to table


In [12]:
try:
    cur.execute("SELECT COUNT(*) FROM songs")
    new_no_of_row = cur.fetchone()
    print(new_no_of_row, " rows in the song_dataset table")
except psycopg2.Error as e:
    print("Error while creating table");
    print(e)

(71,)  rows in the song_dataset table


In [ ]:
# for index in range(len(song_data)):
#     try:
#         cur.execute(song_table_insert, song_data[index])
#         conn.commit()
#     except psycopg2.Error as e:
#         print(e)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]


In [14]:
"""Remove values with same artist ID, keeps last record believing that is the most recent""" 
artist_data_cleaned = artist_data.drop_duplicates(subset=["artist_id"], keep='last')

In [15]:
print(artist_data.shape)
print(artist_data_cleaned.shape)

(71, 5)
(69, 5)


In [16]:
uploaded_filepath = generate_filename("artist_table") 
artist_data_cleaned.to_csv(uploaded_filepath, index=False, header=True,sep=',', encoding='utf-8')
new_artist= open(uploaded_filepath)
print('file opened in memory')

file opened in memory


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
copy_sql = '''COPY public.artists FROM STDIN WITH 
CSV
HEADER
DELIMITER AS ','
'''
try:
    cur.copy_expert(sql=copy_sql,file=new_artist)
    print("file copied to table")
except psycopg2.Error as e:
    print("could not copy file "+ uploaded_filepath+ " to the table\n")
    print(e)

file copied to table


In [ ]:
# for index in range(len(artist_data)):
#     try:
#         cur.execute(artist_table_insert, artist_data)
#         conn.commit()
#     except psycopg2.Error as e:
#         print(e)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = get_files("data/log_data")

In [ ]:
filepath = log_files[0]
filepath

In [20]:
log_df = pd.DataFrame()
for i in range(len(log_files)):
    filepath = log_files[i]
    df = pd.read_json(filepath, lines=True)
    log_df = log_df.append(df)
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
log_df = log_df[log_df['page']=="NextSong"]
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [22]:
t = pd.to_datetime(log_df['ts'],  unit='ms')
t.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [23]:
time_data = []
for datetim in t:
    time_data.append([datetim, datetim.hour, datetim.day, datetim.week, datetim.month, datetim.year, datetim.day_name()])
column_labels = ()

In [24]:
time_df = pd.DataFrame(time_data, columns=["datetime","hour","day","week","month","year","weekday"])
time_df.head()

,datetime,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,Thursday
1,2018-11-01 21:05:52.796,21,1,44,11,2018,Thursday
2,2018-11-01 21:08:16.796,21,1,44,11,2018,Thursday
3,2018-11-01 21:11:13.796,21,1,44,11,2018,Thursday
4,2018-11-01 21:17:33.796,21,1,44,11,2018,Thursday


In [25]:
time_data_cleaned = time_df.drop_duplicates(subset=["datetime"], keep='first')

In [26]:
print(time_df.shape)
print(time_data_cleaned.shape)

(6820, 7)
(6813, 7)


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [27]:
try:
    for i, row in time_data_cleaned.iterrows():
        cur.execute(time_table_insert, list(row))
        #conn.commit()
except psycopg2.Error as e:
    print("table insertion error: ")
    print(e)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [28]:
user_df = log_df[['userId','firstName','lastName','gender','level']]

In [61]:
user_df['userId'] = user_df['userId'].astype(int)

C:\Users\MOTUNR~1\AppData\Local\Temp/ipykernel_7228/2737544736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_df['userId'] = user_df['userId'].astype(int)


In [62]:
#user_df['userId'] = user_df['userId'].apply(lambda x: x.str.strip())
user_df.dtypes

userId        int32
firstName    object
lastName     object
gender       object
level        object
dtype: object

In [63]:
"""Remove values with same user ID, keeps last record believing that is the most recent""" 
user_data_cleaned = user_df.drop_duplicates(subset=['userId'],keep='last')
user_data_cleaned.shape

(96, 5)

In [64]:
user_df['userId'] = user_df['userId'].astype(str)


C:\Users\MOTUNR~1\AppData\Local\Temp/ipykernel_7228/1835240198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_df['userId'] = user_df['userId'].astype(str)


In [65]:
user_data_cleaned.shape

(96, 5)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [49]:
uploaded_filepath = generate_filename("_user_table") 
user_data_cleaned.to_csv(uploaded_filepath, index=False, header=True,sep=',', encoding='utf-8')
new_users = open(uploaded_filepath)
print('file opened in memory')

file opened in memory


In [43]:
copy_sql = '''COPY public.users FROM STDIN WITH 
CSV
HEADER
DELIMITER AS ','
'''
try:
    cur.copy_expert(sql=copy_sql,file=new_users)
    print("file copied to table")
except psycopg2.Error as e:
    print("could not copy file "+ uploaded_filepath+ " to the table\n")
    print(e)

could not copy file uploaded_data\05292022113218486312_user_table.csv to the table

duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(7) already exists.
CONTEXT:  COPY users, line 44



In [66]:
try:
    for i, row in user_data_cleaned.iterrows():
        cur.execute(user_table_insert, row)
        conn.commit()
except psycopg2.Error as e:
    print("table insertion error: ")
    print(e)

userId            3
firstName     Isaac
lastName     Valdez
gender            M
level          free
Name: 170, dtype: object
userId            84
firstName    Shakira
lastName        Hunt
gender             F
level           free
Name: 97, dtype: object
userId            20
firstName      Aiden
lastName     Ramirez
gender             M
level           paid
Name: 12, dtype: object
userId           27
firstName    Carlos
lastName     Carter
gender            M
level          free
Name: 214, dtype: object
userId           59
firstName      Lily
lastName     Cooper
gender            F
level          free
Name: 86, dtype: object
userId          99
firstName      Ann
lastName     Banks
gender           F
level         free
Name: 148, dtype: object
userId           34
firstName    Evelin
lastName      Ayala
gender            F
level          free
Name: 316, dtype: object
userId           62
firstName    Connar
lastName     Moreno
gender            M
level          free
Name: 338, dtype: objec

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [67]:
for index, row in log_df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        #print(results)
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [68]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.